In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
path_GR = "../Resources/raw/OxCGRT_latest(Aug31,2020).csv"   # Government Response
path_CD = "../Resources/raw/owid-covid-data(Aug31,2020).csv"   # COVID Data

In [3]:
data_GR = pd.read_csv(path_GR)    #Government Response
data_CD = pd.read_csv(path_CD)

C:\Users\muzzn\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_GR = data_GR[['Date','CountryCode']]
stringency_columns = [ x for x in data_GR if re.search("C\d|H1",x)]

for x in stringency_columns:
    df_GR[x] = data_GR[x]
    
df_GR

C:\Users\muzzn\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Date,CountryCode,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,C5_Close public transport,C5_Flag,C6_Stay at home requirements,C6_Flag,C7_Restrictions on internal movement,C7_Flag,C8_International travel controls,H1_Public information campaigns,H1_Flag
0,20200101,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN
1,20200102,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN
2,20200103,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN
3,20200104,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN
4,20200105,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58799,20200827,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58800,20200828,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58801,20200829,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58802,20200830,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_GR.isnull().sum()

Date                                        0
CountryCode                                 0
C1_School closing                        1763
C1_Flag                                 22666
C2_Workplace closing                     1887
C2_Flag                                 24946
C3_Cancel public events                  1820
C3_Flag                                 21870
C4_Restrictions on gatherings            1853
C4_Flag                                 24858
C5_Close public transport                1695
C5_Flag                                 37073
C6_Stay at home requirements             1853
C6_Flag                                 28354
C7_Restrictions on internal movement     1741
C7_Flag                                 28815
C8_International travel controls         1841
H1_Public information campaigns          2299
H1_Flag                                 13060
dtype: int64

In [6]:
for x in df_GR.columns:
    if re.search("C\d|H1",x):
        print(df_GR[x].value_counts())

3.0    26644
0.0    20903
2.0     7762
1.0     1732
Name: C1_School closing, dtype: int64
1.0    31024
0.0     5114
Name: C1_Flag, dtype: int64
0.0    23059
2.0    19227
3.0     8210
1.0     6421
Name: C2_Workplace closing, dtype: int64
1.0    27436
0.0     6422
Name: C2_Flag, dtype: int64
2.0    30281
0.0    20050
1.0     6653
Name: C3_Cancel public events, dtype: int64
1.0    32840
0.0     4094
Name: C3_Flag, dtype: int64
0.0    23005
4.0    17065
3.0    12451
2.0     3621
1.0      809
Name: C4_Restrictions on gatherings, dtype: int64
1.0    29289
0.0     4657
Name: C4_Flag, dtype: int64
0.0    35378
1.0    13465
2.0     8266
Name: C5_Close public transport, dtype: int64
1.0    14627
0.0     7104
Name: C5_Flag, dtype: int64
0.0    26501
1.0    14382
2.0    13630
3.0     2438
Name: C6_Stay at home requirements, dtype: int64
1.0    23740
0.0     6710
Name: C6_Flag, dtype: int64
0.0    27074
2.0    18406
1.0    11583
Name: C7_Restrictions on internal movement, dtype: int64
1.0    20850


In [7]:
df_GR = df_GR.fillna(0)

In [8]:
df_CD_1 = data_CD[["date","iso_code","total_cases","total_deaths"]]
df_CD_2 = data_CD[["date","iso_code","new_cases","new_deaths"]]
df_CD_2 = df_CD_2.fillna(0)

In [9]:
df_CD = df_CD_1.merge(df_CD_2,on=["date","iso_code"])

In [10]:
df_CD.head()

,date,iso_code,total_cases,total_deaths,new_cases,new_deaths
0,2020-03-13,ABW,2.0,0.0,2.0,0.0
1,2020-03-19,ABW,NaN,NaN,0.0,0.0
2,2020-03-20,ABW,4.0,0.0,2.0,0.0
3,2020-03-21,ABW,NaN,NaN,0.0,0.0
4,2020-03-22,ABW,NaN,NaN,0.0,0.0


In [11]:
codes = df_CD["iso_code"].unique()

iso_dfs = []

total_cases_col = 2
tota_deaths_col = 3

for code in codes:
    iso_df = df_CD[df_CD["iso_code"] == code]
    if np.isnan(iso_df.iloc[0,total_cases_col]):
        iso_df.iloc[0,total_cases_col] = 0
    if np.isnan(iso_df.iloc[0,tota_deaths_col]):
        iso_df.iloc[0,tota_deaths_col] = 0
    
    iso_df = iso_df.fillna(method='ffill')
    
    iso_dfs.append(iso_df)

df_CD = pd.DataFrame()

for iso_df in iso_dfs:
    df_CD = df_CD.append(iso_df)
    
df_CD.isnull().sum()

C:\Users\muzzn\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


date            0
iso_code        0
total_cases     0
total_deaths    0
new_cases       0
new_deaths      0
dtype: int64

In [12]:
df_CD

,date,iso_code,total_cases,total_deaths,new_cases,new_deaths
0,2020-03-13,ABW,2.0,0.0,2.0,0.0
1,2020-03-19,ABW,2.0,0.0,0.0,0.0
2,2020-03-20,ABW,4.0,0.0,2.0,0.0
3,2020-03-21,ABW,4.0,0.0,0.0,0.0
4,2020-03-22,ABW,4.0,0.0,0.0,0.0
...,...,...,...,...,...,...
40498,2020-08-27,OWID_WRL,24201449.0,825936.0,276699.0,6297.0
40499,2020-08-28,OWID_WRL,24484672.0,832011.0,283223.0,6075.0
40500,2020-08-29,OWID_WRL,24761119.0,837466.0,276447.0,5455.0
40501,2020-08-30,OWID_WRL,25029850.0,843154.0,268731.0,5688.0


In [13]:
df_GR.columns

Index(['Date', 'CountryCode', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'H1_Public information campaigns',
       'H1_Flag'],
      dtype='object')

In [14]:
df_CD.columns

Index(['date', 'iso_code', 'total_cases', 'total_deaths', 'new_cases',
       'new_deaths'],
      dtype='object')

In [15]:
df_GR = df_GR.rename(columns={"Date":"date","CountryCode":"iso_code"})

In [16]:
def formatDate(dt):
    d_str = str(dt)
    year = d_str[0:4]
#     print(year)
    month = d_str[4:6]
#     print(month)
    day = d_str[6:8]
#     print(day)
    dt_str = year + "-" + month + "-" + day
    
    return dt_str

In [17]:
type(df_GR["date"][0])

numpy.int64

In [18]:
formatDate(df_GR["date"][0])

'2020-01-01'

In [19]:
df_GR["date"] = df_GR["date"].apply(lambda x:formatDate(x))

In [20]:
df_GR.head()

,date,iso_code,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,C5_Close public transport,C5_Flag,C6_Stay at home requirements,C6_Flag,C7_Restrictions on internal movement,C7_Flag,C8_International travel controls,H1_Public information campaigns,H1_Flag
0,2020-01-01,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-02,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-03,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-04,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-05,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
df_CD.head()

,date,iso_code,total_cases,total_deaths,new_cases,new_deaths
0,2020-03-13,ABW,2.0,0.0,2.0,0.0
1,2020-03-19,ABW,2.0,0.0,0.0,0.0
2,2020-03-20,ABW,4.0,0.0,2.0,0.0
3,2020-03-21,ABW,4.0,0.0,0.0,0.0
4,2020-03-22,ABW,4.0,0.0,0.0,0.0


In [22]:
final_df = df_GR.merge(df_CD,on=["date","iso_code"])
final_df

,date,iso_code,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,...,C6_Flag,C7_Restrictions on internal movement,C7_Flag,C8_International travel controls,H1_Public information campaigns,H1_Flag,total_cases,total_deaths,new_cases,new_deaths
0,2020-03-13,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0
1,2020-03-19,ABW,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,2.0,1.0,2.0,0.0,0.0,0.0
2,2020-03-20,ABW,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,2.0,1.0,4.0,0.0,2.0,0.0
3,2020-03-21,ABW,3.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,4.0,2.0,1.0,4.0,0.0,0.0,0.0
4,2020-03-22,ABW,3.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,4.0,2.0,1.0,4.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48523,2020-08-27,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6251.0,179.0,55.0,13.0
48524,2020-08-28,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6292.0,189.0,41.0,10.0
48525,2020-08-29,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6388.0,195.0,96.0,6.0
48526,2020-08-30,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6406.0,196.0,18.0,1.0


In [23]:
final_df.to_csv("../Resources/StringencyColumns.csv")

In [24]:
!pwd

The system cannot find the path specified.
'pwd' is not recognized as an internal or external command,
operable program or batch file.


NameError: name 'plt' is not defined